In [5]:
import pandas as pd

## Beispieldaten analysieren

In [7]:
events = pd.read_csv('../data/de_challenge_sample_data.csv')

In [9]:
events.columns

Index(['Unnamed: 0', '$schema', 'id', 'type', 'namespace', 'title', 'comment',
       'timestamp', 'user', 'bot', 'minor', 'patrolled', 'server_url',
       'server_name', 'server_script_path', 'wiki', 'parsedcomment',
       'meta_domain', 'meta_uri', 'meta_request_id', 'meta_stream',
       'meta_topic', 'meta_dt', 'meta_partition', 'meta_offset', 'meta_id',
       'length_old', 'length_new', 'revision_old', 'revision_new'],
      dtype='object')

In [13]:
events.head(5)

,Unnamed: 0,$schema,id,type,namespace,title,comment,timestamp,user,bot,...,meta_stream,meta_topic,meta_dt,meta_partition,meta_offset,meta_id,length_old,length_new,revision_old,revision_new
0,0,/mediawiki/recentchange/1.0.0,1389063095,edit,0,Q104971167,/* wbeditentity-update-languages-short:0||nl *...,1611249080,Edoderoobot,True,...,mediawiki.recentchange,eqiad.mediawiki.recentchange,2021-01-21T17:11:20Z,0,2887301727,a62392d6-25d3-405c-9d6c-54956eb60a52,3781,3860,1345581201,1345601868
1,0,/mediawiki/recentchange/1.0.0,116494285,edit,0,Acanthastrea erythraea,[[Wikipedia:Geen samenvatting|Verwijst door]] ...,1611249078,Kvdrgeus,False,...,mediawiki.recentchange,eqiad.mediawiki.recentchange,2021-01-21T17:11:18Z,0,2887301728,f6acf301-d987-4d7b-85fe-d2c3cb486ffb,866,65,48016794,58091518
2,0,/mediawiki/recentchange/1.0.0,119689610,edit,0,神楽シリーズ,/* シリーズ作品 */,1611249080,Zaco9876,False,...,mediawiki.recentchange,eqiad.mediawiki.recentchange,2021-01-21T17:11:20Z,0,2887301732,6f36af6f-f84b-421c-bd5d-ebc2eef54f0f,22851,22855,81442784,81442790
3,0,/mediawiki/recentchange/1.0.0,283652737,edit,0,Nana & Kaoru,Aggiornamento numero volumi pubblicati.,1611249080,Pazio Paz,False,...,mediawiki.recentchange,eqiad.mediawiki.recentchange,2021-01-21T17:11:20Z,0,2887301735,ffc41bdc-1bb8-4c1b-af88-cb066687d441,31030,31030,118007674,118137110
4,0,/mediawiki/recentchange/1.0.0,1389063094,edit,0,Q37914681,/* wbsetdescription-add:1|bn */ রাশিয়ার একটি ...,1611249080,Tahmid02016,False,...,mediawiki.recentchange,eqiad.mediawiki.recentchange,2021-01-21T17:11:20Z,0,2887301737,10a1a1da-da51-4991-b248-4f3365d5aaf2,6582,6688,898245132,1345601865


schema: https://github.com/wikimedia/mediawiki-event-schemas/blob/master/jsonschema/mediawiki/recentchange/1.0.0.yaml


wichtig für streaming: timestamp column (unix timestamp)

es müssen später changes nach country gegrouped werden zur analyse

In [14]:
events.iloc[0,:]

Unnamed: 0                                                            0
$schema                                   /mediawiki/recentchange/1.0.0
id                                                           1389063095
type                                                               edit
namespace                                                             0
title                                                        Q104971167
comment               /* wbeditentity-update-languages-short:0||nl *...
timestamp                                                    1611249080
user                                                        Edoderoobot
bot                                                                True
minor                                                             False
patrolled                                                          True
server_url                                     https://www.wikidata.org
server_name                                            www.wikid

In [17]:
events[['server_name', 'meta_domain', ]].value_counts()

server_name            meta_domain          
www.wikidata.org       www.wikidata.org         440
commons.wikimedia.org  commons.wikimedia.org    142
en.wikipedia.org       en.wikipedia.org         126
fr.wikipedia.org       fr.wikipedia.org          21
it.wikipedia.org       it.wikipedia.org          20
                                               ... 
hr.wikipedia.org       hr.wikipedia.org           1
hi.wikisource.org      hi.wikisource.org          1
fr.wiktionary.org      fr.wiktionary.org          1
fr.wikisource.org      fr.wikisource.org          1
af.wikipedia.org       af.wikipedia.org           1
Length: 63, dtype: int64

In [19]:
(events['meta_domain'] == 'de.wikipedia.org').value_counts()

False    984
True      16
Name: meta_domain, dtype: int64

--> deutsche wikipedia kann über server_name bzw meta_domain identifiziert werden

## TODO
1. kafka Umgebung aufsetzen
2. python container aufsetzen, der mit kafka-python die Beispieldaten importiert und emittiert
3. pyspark container aufsetzen, der vom topic einliest und aggregationen vornimmt
4. postgres datenbank aufbauen um die ergebnisse abzuspeichern

In [23]:
events.to_dict(orient='records')

[{'Unnamed: 0': 0,
  '$schema': '/mediawiki/recentchange/1.0.0',
  'id': 1389063095,
  'type': 'edit',
  'namespace': 0,
  'title': 'Q104971167',
  'comment': '/* wbeditentity-update-languages-short:0||nl */ nl-description, [[User:Edoderoobot/Set-nl-description|python code]] - chemische samenstelling',
  'timestamp': 1611249080,
  'user': 'Edoderoobot',
  'bot': True,
  'minor': False,
  'patrolled': True,
  'server_url': 'https://www.wikidata.org',
  'server_name': 'www.wikidata.org',
  'server_script_path': '/w',
  'wiki': 'wikidatawiki',
  'parsedcomment': '\u200e<span dir="auto"><span class="autocomment">Changed label, description and/or aliases in nl: </span></span> nl-description, <a href="/wiki/User:Edoderoobot/Set-nl-description" title="User:Edoderoobot/Set-nl-description">python code</a> - chemische samenstelling',
  'meta_domain': 'www.wikidata.org',
  'meta_uri': 'https://www.wikidata.org/wiki/Q104971167',
  'meta_request_id': 'YAm1uApAIIEAACcb76EAAAAA',
  'meta_stream': 'me